In [1]:
import numpy as np
import numpy.linalg as la
import scipy.linalg as sla
import tensors.synthetic_tensors as synthetic_tensors
import backend.numpy_ext as tenpy
from CPD.common_kernels import get_residual, compute_lin_sys

import matplotlib.pyplot as plt

# Naive NLS Implementation

In [2]:
def jacobian3(A):
    J = np.zeros((Q,order*s*R))
    for i in range(order):
        offset1 = i*s*R
        for j in range(R):
            offset2 = j*s
            start = offset1+offset2
            end = start + s
            if i==0:
                J[:,start:end] = np.kron(np.identity(s),np.kron(A[1][:,j],A[2][:,j])).T
            elif i==1:
                J[:,start:end] = np.kron(A[0][:,j],np.kron(np.identity(s),A[2][:,j])).T
            elif i==2:
                J[:,start:end] = np.kron(A[0][:,j],np.kron(A[1][:,j],np.identity(s))).T
    return J

def F(T,A):
    f = (T - np.einsum("ir,jr,kr->ijk",A[0],A[1],A[2])).reshape(-1)
    return f


In [3]:
def gradient(T,A):
    g = np.zeros(order*s*R)
    G = []
    
    TC = tenpy.einsum("ijk,ka->ija",T,A[2])
    M1 = tenpy.einsum("ija,ja->ia",TC,A[1])
    G1 = -1*M1 + np.dot(A[0],compute_lin_sys(tenpy,A[1],A[2],0))
    G.append(G1)
    
    M2 = tenpy.einsum("ija,ia->ja",TC,A[0])
    G2 = -1*M2 + np.dot(A[1],compute_lin_sys(tenpy,A[0],A[2],0))
    G.append(G2)
    
    M3 = tenpy.einsum("ijk,ia,ja->ka",T,A[0],A[1])
    G3 = -1*M3 + np.dot(A[2],compute_lin_sys(tenpy,A[0],A[1],0))
    G.append(G3)
    
    for i in range(order):
        offset1 = i*s*R
        for j in range(R):
            offset2 = j*s
            start = offset1 + offset2
            end = start + s
            g[start:end] = G[i][:,j]
    
    return g

In [4]:
def flatten_A(A):
    a = np.zeros(order*R*s)
    for i in range(order):
        offset1 = i*s*R
        for j in range(R):
            offset2 = j*s
            start = offset1+offset2
            end = start+s
            a[start:end] = A[i][:,j]
    return a

def update_A(A,x):
    for i in range(order):
        offset1 = i*s*R
        for j in range(R):
            offset2 = j*s
            start = offset1+offset2
            end = start+s
            A[i][:,j] += x[start:end]

# Fast NLS Implementation with Block Matrix-Vector Multiplication

In [5]:
# fast Hessian approximation by Gauss-Newton
def compute_coeff(G,n1,r1,n2,r2):
    return np.prod([G[i][r1,r2] for i in range(len(G)) if i!=n1 and i!=n2])
        
def compute_block(A,G,n1,r1,n2,r2):
    if n1 == n2:
        return compute_coeff(G,n1,r1,n2,r2)*np.identity(A[0].shape[0])
    else:
        return compute_coeff(G,n1,r1,n2,r2)*np.outer(A[n1][:,r2],A[n2][:,r1])

def fast_hessian3(A):
    G1 = A[0].T.dot(A[0])
    G2 = A[1].T.dot(A[1])
    G3 = A[2].T.dot(A[2])
    G = [G1,G2,G3]
    N = order*s*R
    hessian = np.zeros((N,N))
    
    for n1 in range(order):
        for r1 in range(R):
            startv = n1*R*s + r1*s
            endv = startv + s
            for n2 in range(order):
                for r2 in range(R):
                    starth = n2*R*s + r2*s
                    endh = starth + s
                    hessian[startv:endv,starth:endh] = compute_block(A,G,n1,r1,n2,r2)
    return hessian

def compute_result_block(A,G,n1,r1,n2,r2,x):
    if n1==n2:
        return compute_coeff(G,n1,r1,n2,r2)*x
    else:
        s = compute_coeff(G,n1,r1,n2,r2)*np.inner(A[n2][:,r1],x)
        return s*A[n1][:,r2]

def fast_hessian3_mult(A,x,regu=1):
    ret = regu*x
    G = []
    for i in range(len(A)):
        G.append(A[i].T.dot(A[i]))
    
    for n1 in range(order):
        for r1 in range(R):
            startv = n1*R*s + r1*s
            endv = startv + s
            for n2 in range(order):
                for r2 in range(R):
                    starth = n2*R*s + r2*s
                    endh = starth + s
                    ret[startv:endv] += compute_result_block(A,G,n1,r1,n2,r2,x[starth:endh])
    return ret


In [63]:
def conjugate_gradient(A,x,b,tol=1e-4,formula="PR"):
    tol = np.max([tol,tol*la.norm(b)])
    r = b - A.dot(x)
    if la.norm(r)<tol:
        return x,0
    p = r
    counter = 0
    while True:
        alpha = np.inner(r,r)/np.inner(p,A.dot(p))
        x += alpha*p
        r_new = r - alpha*A.dot(p)
        if la.norm(r_new)<tol:
            break
        if formula == "PR":
            beta = np.inner(r_new,r_new-r)/np.inner(r,r)
        elif formula == "HS":
            y = r_new - r
            beta = np.inner(r_new,y)/np.inner(y,p)
        else:
            beta = np.inner(r_new,r_new)/np.inner(r,r)
        p = r_new + beta*p
        r = r_new
        counter += 1
    print("conjugate gradient took ",counter," iteration(s).")
    return x,counter

def preconditioned_conjugate_gradient(A,x,b,M,tol=1e-4,formula="PR"):
    tol = np.max([tol,tol*la.norm(b)])
    r = b - A.dot(x)
    counter = 0
    if la.norm(r)<tol:
        return x,counter
    z = M.dot(r)
    p = z
    while True:
        alpha = np.inner(r,z)/np.inner(p,A.dot(p))
        x += alpha*p
        r_new = r - alpha*A.dot(p)
        if la.norm(r_new)<tol: ## need to add max iteration
            break
        z_new = M.dot(r_new)
        if formula == "PR":
            beta = np.inner(z_new,r_new-r)/np.inner(z,r)
        elif formula == "HS":
            y=z_new-z
            beta = np.inner(z_new,y)/np.inner(y,p)
        else:
            beta = np.inner(z_new,r_new)/np.inner(z,r)
        p = z_new + beta*p
        r = r_new
        z = z_new
        counter += 1
    return x,counter

def preconditioned_conjugate_gradient2(A,x,b,M,tol=1e-4,formula="PR"):
    r = b - A.dot(x)
    counter = 0
    if la.norm(r)<tol:
        return x,counter
    z = naive_block_GS(A,3,R,r) #M.dot(r)
    p = z
    while True:
        alpha = np.inner(r,z)/np.inner(p,A.dot(p))
        x += alpha*p
        r_new = r - alpha*A.dot(p)
        if la.norm(r_new)<tol*la.norm(x): ## need to add max iteration
            break
        z_new = naive_block_GS(A,3,R,r_new) #M.dot(r_new)
        if formula == "PR":
            beta = np.inner(z_new,r_new-r)/np.inner(z,r)
        else:
            beta = np.inner(z_new,r_new)/np.inner(z,r)
        p = z_new + beta*p
        r = r_new
        z = z_new
        counter += 1
    print("conjugate gradient took ",counter," iteration(s).")
    return x,counter


def fast_conjugate_gradient():
    return 

In [51]:
def symmetric_solve(H,x):
    L = la.cholesky(H)
    Y = sla.solve_triangular(L,x,trans=0,lower=True)
    X = sla.solve_triangular(L,Y,trans=1,lower=True)
    return X

def backward_symmetric_solve(H,x):
    X = symmetric_solve(H,x.T)
    return X.T

def naive_block_Jacobi(H,order,stride):
    M = np.zeros(H.shape)
    for i in range(order):
        start = i*stride
        end = start + stride
        M[start:end,start:end] = H[start:end,start:end]
    L = la.cholesky(M)
    Y = sla.solve_triangular(L,np.identity(H.shape[0]),trans=0,lower=True)
    X = sla.solve_triangular(L,Y,trans=1,lower=True)
    return X

def naive_block_GS(H,order,stride,x):
    ret = np.copy(x)
    for n in range(order):
        for p in range(n+1):
            startv = n*stride
            endv = startv + stride
            starth = p*stride
            endh = starth + stride
            M = H[startv:endv,starth:endh]
            if n==p:
                ret[startv:endv] = symmetric_solve(M,x[startv:endv])
            else:
                ret[startv:endv] -= M.dot(x[startv:endv])
    return ret

def naive_block_SGS(H,order,stride,x):
    ret = np.copy(x)
    for n in range(order):
        for p in range(n+1):
            startv = n*stride
            endv = startv + stride
            starth = p*stride
            endh = starth + stride
            M = H[startv:endv,starth:endh]
            if n==p:
                ret[startv:endv] = symmetric_solve(M,x[startv:endv])
            else:
                ret[startv:endv] -= M.dot(x[startv:endv])
    for n in range(order):
        startv = n*stride
        endv = startv + stride
        M = H[startv:endv,startv:endv]
        ret[startv:endv] = M.dot(ret[startv:endv])
    for n in range(order,0,-1):
        for p in range(order,n-1,-1):
            startv = (n-1)*stride
            endv = startv + stride
            starth = (p-1)*stride
            endh = starth + stride
            M = H[startv:endv,starth:endh]
            if n==p:
                ret[startv:endv] = symmetric_solve(M,ret[startv:endv])
            else:
                ret[startv:endv] -= M.dot(ret[startv:endv])
    return ret

def naive_SGS(H):
    I = np.identity(H.shape[0])
    Y = sla.solve_triangular(H,I,trans=0,lower=True)
    D = np.diag(H)
    Y = np.einsum("i,ij->ij",D,Y)
    Y = sla.solve_triangular(H,Y,trans=0,lower=False)
    return Y

def naive_GS(H):
    I = np.identity(H.shape[0])
    Y = sla.solve_triangular(H,I,trans=0,lower=True)
    return Y

def Jacobi_preconditioner(H):
    s1 = np.sum(np.diag(H))
    P = np.diag(1/np.diag(H))
    s2 = np.sum(np.diag(P))
    return P

def block_LU(A,stride):
    L = np.zeros(A.shape)
    U = np.zeros(A.shape)
    A11 = A[:stride,:stride]
    
    L[:stride,:stride] = np.identity(stride)
    U[:stride,:] = A[:stride,:]
    U12 = A[:stride,stride:]
    L[stride:,:stride] = backward_symmetric_solve(A11,A[stride:,:stride])
    L21 = L[stride:,:stride]
    if A.shape[0] == stride:
        return L,U
    else:
        L[stride:,stride:],U[stride:,stride:] = block_LU(A[stride:,stride:]-L21.dot(U12),stride)
        return L,U

def naive_block_diag_perturbative_preconditioner(H,order,stride):
    A_inv = naive_block_Jacobi(H,order,stride)
    B = np.zeros(H.shape)
    for n in range(order):
        for p in range(order):
            startv = n*stride
            endv = startv + stride
            starth = p*stride
            endh = starth + stride
            if n!=p:
                B[starth:endh,startv:endv] = -1*H[starth:endh,startv:endv]
    return 2*A_inv - A_inv.dot(H.dot(A_inv))

def naive_diag_perturbative_preconditioner(H):
    s1 = np.sum(np.diag(H))
    s2 = np.sum(1/np.diag(H))
    D = np.diag(np.diag(H))
    B = D - H
    D_inv = np.diag(1/np.diag(H))/(s1*s2)
    return 2*D_inv - D_inv.dot(H.dot(D_inv))
    
def identity_like_iter_preconditioner(H):
    Fnorm = la.norm(H,ord='fro')
    print("Fnorm is ",Fnorm)
    V0 = np.identity(H.shape[0])/Fnorm
    return 2*V0 - H/Fnorm**2

def identity_like_iter_preconditioner2(H):
    s = la.eigvalsh(H)[-1]/2+1
    print("sigma/2 is ",s)
    s2 = power_iteration(H,np.random.random(H.shape[0]))
    #print("power iteration gives ",s2)
    #print("power iteration estimate sigma/2 is ",s2/2)
    s2 = s2/2
    V0 = np.identity(H.shape[0])/s2
    return 2*V0 - H/s2**2

def transpose_iter_preconditioner(H):
    d = np.max(np.sum(H,axis=1))
    V0 = H/d**2
    return 2*V0 + V0.dot(H.dot(V0))

def power_iteration(H,x,iteration=1):
    x = x/la.norm(x)
    for i in range(iteration):
        x = H.dot(x)
        x = x/la.norm(x)
    return np.inner(x,H.dot(x))

In [78]:
#naive NLS implementation for order 3 CP decomposition
order = 3
s = 4
R = 7
sp_frac = 1
iteration = 50

[T,O] = synthetic_tensors.init_rand(tenpy,order,s,R,sp_frac)
A = []
for i in range(T.ndim):
    A.append(tenpy.random((T.shape[i],R)))

Q = s**order

In [79]:
res = get_residual(tenpy,T,A)
print("Start residual is ",res)
x = flatten_A(A)
print("x shape is",x.shape)
a = 0
for i in range(iteration):
    J = jacobian3(A)
    JT = np.transpose(J)
    H = np.dot(JT,J)
    regu = 1/(i+1)
    H += regu*np.identity(J.shape[1])
    #H = fast_hessian3(A,regu)
    
    b = -1*gradient(T,A)
    #print("[",i,"] iteration gradient norm is ",la.norm(b))
    M = naive_block_Jacobi(H,order,s*R)
    #M = naive_diag_perturbative_preconditioner(H)
    #M = identity_like_iter_preconditioner2(H)
    #M = Jacobi_preconditioner(H)
    #f = F(T,A)
    #b = np.dot(JT,f)
    #L = la.cholesky(H)
    #y = sla.solve_triangular(L,b,trans=0,lower=True)
    #x = sla.solve_triangular(L,y,trans=1,lower=True)
    #x,c = conjugate_gradient(H,x,b,formula="PR")
    x,c = preconditioned_conjugate_gradient(H,x,b,M,formula="PR")
    print("[",i,"] conjugate gradient took ",c," iteration(s).")
    #x,c = preconditioned_conjugate_gradient2(H,x,b,M)
    a += c
    update_A(A,x)
    res = get_residual(tenpy,T,A)
    print("[",i,"] iteration residual is ",res)
    if res<1e-4:
        break
print("Total number of CG iterations is ",a)

('Residual computation took', 0.00025582313537597656, 'seconds')
Start residual is  4.727867857099142
x shape is (84,)
[ 0 ] conjugate gradient took  15  iteration(s).
('Residual computation took', 5.841255187988281e-05, 'seconds')
[ 0 ] iteration residual is  1.3659613484045996
[ 1 ] conjugate gradient took  23  iteration(s).
('Residual computation took', 0.000102996826171875, 'seconds')
[ 1 ] iteration residual is  0.7799629266609043
[ 2 ] conjugate gradient took  28  iteration(s).
('Residual computation took', 0.00011658668518066406, 'seconds')
[ 2 ] iteration residual is  0.5014164839052382
[ 3 ] conjugate gradient took  28  iteration(s).
('Residual computation took', 0.00011372566223144531, 'seconds')
[ 3 ] iteration residual is  0.32367504572096
[ 4 ] conjugate gradient took  31  iteration(s).
('Residual computation took', 7.343292236328125e-05, 'seconds')
[ 4 ] iteration residual is  0.22766067074733876
[ 5 ] conjugate gradient took  32  iteration(s).
('Residual computation took

# Fast NLS Implementation with Tensor Contraction

In [26]:
def compute_coefficient_matrix(G,n1,n2):
    ret = np.ones(G[0].shape)
    for i in range(len(G)):
        if i!=n1 and i!=n2:
            ret = np.einsum("ij,ij->ij",ret,G[i])
    return ret

def compute_coefficient_matrix_matrix(G):
    N = len(G)
    ret = [[None for i in range(N)] for i in range(N)]
    for n in range(N):
        for p in range(n,N):
            M = compute_coefficient_matrix(G,n,p)
            ret[n][p] = M
            if n!=p:
                ret[p][n] = M
    return ret

def fast_hessian_contract(A,X,regu=1):
    N = len(A)
    ## Preprocessing step: should be moved outside of contraction 
    G = []
    for mat in A:
        G.append(mat.T.dot(mat))
    
    ret = []
    for n in range(N):
        for p in range(N):
            ## Computation of M should be done outside of contraction
            M = compute_coefficient_matrix(G,n,p)
            if n==p:
                Y = np.einsum("iz,zr->ir",X[p],M)
            else:
                Y = np.einsum("iz,zr,jr,jz->ir",A[n],M,A[p],X[p])
            if p==0:
                ret.append(Y)
            else:
                ret[n] += Y
                
    for i in range(N):
        ret[i] += regu*X[i]
    return ret

In [29]:
X = [np.random.random((s,R)) for i in range(order)]
x = flatten_A(X)
J = jacobian3(A)
JT = np.transpose(J)
H = np.dot(JT,J)
r1 = H.dot(x)
r2 = flatten_A(fast_hessian_contract(A,X))
print(np.isclose(r1,r2))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]


# Perturbative Algorithm for NLS